https://spark.apache.org/mllib/

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/24 19:26:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/24 19:26:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/24 19:26:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/24 19:26:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/24 19:26:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/24 19:26:44 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [2]:
## read the dataset
training=spark.read.csv('test.csv',header=True,inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

## Dataset is divided into dependent and Independent features

- [Age, Experience]------>new feature ------> independent feature
- [salary]-------> dependent feature
- [Name] -----> not needed

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [9]:
output=featureassembler.transform(training)

In [10]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data=output.select("Independent Features", "Salary")

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



## Dataset is divided into train-test split method

In [14]:
from pyspark.ml.regression import LinearRegression

In [15]:
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Features", labelCol='Salary')
regressor=regressor.fit(train_data)
                           

22/10/24 19:47:04 WARN Instrumentation: [bd802e74] regParam is zero, which might cause numerical instability and overfitting.
22/10/24 19:47:05 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/24 19:47:05 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/10/24 19:47:05 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [16]:
### Coefficients
regressor.coefficients

DenseVector([-102.53, 1688.6818])

In [17]:
### Intercepts
regressor.intercept

16470.03994673731

In [18]:
### prediction
pred_results=regressor.evaluate(test_data)

In [20]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [30.0,8.0]| 25000|26903.59520639148|
+--------------------+------+-----------------+



In [21]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1903.595206391481, 3623674.709796625)